In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fraud Detection Workshops Starter Notebook
## Predicting Fraudulent Transactions

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?name=FinTech%3A%20Credit%20Risk&download_url=https%3A%2F%2Fgist.githubusercontent.com%2Foliviervg1%2Fa8f94d673be2aa7c6cffe384a582c9cc%2Fraw%2Fcredit_risk_bqml_python.ipynb&url=https%3A%2F%2Fgist.github.com%2Foliviervg1%2Fa8f94d673be2aa7c6cffe384a582c9cc">
      <img src="https://cloud.google.com/images/products/ai/ai-solutions-icon.svg" alt="Vertex AI Notebooks">Run on Vertex AI Notebooks</a>
  </td>
</table>

- [Background](#background)
- [Setup BigQuery in Jupyter](#setup)
- [BigQuery ML basics](#basics)
  - [Select BigQuery data from Python](#select)
  - [Train a BigQuery ML model](#train)
  - [Check models currently being trained on BigQuery](#currentlytraining)
  - [Check if a model has finished training](#finishedtraining)
  - [Evaluate a BigQuery ML model](#evaluate)
  - [Compare multiple models](#compare)
  - [ML.FEATURE_INFO to check input feature information](#featinfo)
  - [Make predictions using BigQuery ML](#predictions)
  - [Export predictions as a CSV file](#exportcsv)
- [Feature engineering](#feateng)
  - [EXTRACT: Extract dates and time](#extractdates)
  - [REGEXP_EXTRACT_ALL: Extract words from a STRING column](#extractwords)
  - [ML.QUANTILE_BUCKETIZE: Bucketize your numeric feature into bins](#bucketize)
  - [ML.POLYNOMIAL_EXPAND: Combine numeric features](#polynomialexpand)
  - [ML.FEATURE_CROSS: Combine categorical features](#featurecross)
- [Feature selection](#featselection)
  - [ML.FEATURE_IMPORTANCE for Boosted Tree models](#featimportance)
- [Submit Your Results!](#submit)

### Background <a id="background"></a>

In this lab you will be attempting to detect fraudulent transactions using a synthetic dataset. The dataset includes 50k customers and 5k payment terminals. You will be given ~3.4M labeled transactions (fraud 0|1) for training and ~70k unlabeled transactions for evaluation. The locations of the terminals and cutomers are within the same 100x100 cartesian grid.

You will be provided the following four tables in the SOURCE_DATASET:
- customers
  - 50,000 customers along with their residence location on a 100x100 grid
    - CUSTOMER_ID - STRING - Unique identifier of customer
    - x_customer_id - FLOAT - x-coordinate of customer in (0,100)
    - y_customer_id - FLOAT - y-coordinate of customer in (0,100)
- terminals
  - 5,000 terminals along with their locations on a 100x100 grid
    - TERMINAL_ID - STRING - Unique identifier of terminal
    - x_terminal_id - FLOAT - x-coordinate of terminal in (0,100)
    - y_terminal_id - FLOAT - y-coordinate of terminal in (0,100)
- train
  - ~3.4M labeled transactions from __Jan-Nov__ including the customer ID and terminal ID, time of the transaction, amount and fraud/not-fraud.
    - TX_ID - Unique ID of transaction
    - TX_TS - TIMESTAMP - Timestamp of transaction
    - CUSTOMER_ID - STRING - Unique identifier of customer
    - TERMINAL_ID - STRING - Unique identifier of terminal
    - TX_AMOUNT - NUMERIC - Dollar amount of transaction
    - TX_FRAUD - INT - __LABEL__ - 1 if fraudulent, 0 if not.
- test
  - ~70k unlabeled transactions from __Dec 1st-7th__
    - TX_ID - Unique ID of transaction
    - TX_TS - TIMESTAMP - Timestamp of transaction
    - CUSTOMER_ID - STRING - Unique identifier of customer
    - TERMINAL_ID - STRING - Unique identifier of terminal
    - TX_AMOUNT - NUMERIC - Dollar amount of transaction

### High Level Data Diagram

<img src="https://storage.googleapis.com/kaggleworkshops-fraud-detection-public/Kaggle%20Data.svg" width=1000px>

<a id="setup"></a>

### Setup BigQuery in Jupyter <a class="anchor" id="setup"></a>

#### Install some libraries

__*IMPORTANT:*__ After running the following cell, your notebook kernel will restart automatically. After you run it, please comment the code with "#" on each line to avoid restarting your kernel accidentally after that.

In [ ]:
!pip install google-cloud-bigquery google-cloud-bigquery-storage
import IPython

IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

#### Set parameters

For this hackathon you will pull the data from source project `kaggleworkshops`. As you work you will save your data into your team project, in a new dataset that you will create. We will be storing our data in __us-central1__

In [85]:
#Check current project
GCP_PROJECTS = !gcloud config get-value project
print(GCP_PROJECTS)

['euphoric-quanta-238417']


__*NOTE*__: Change below DATASET_ID to a different name for yourself, if you want to work in your own dataset separate from your team's. You can also create multiple datasets if you want, just make sure you are using the correct dataset for whatever query or work you are doing. You may have to manipulate variables in below cells accordingly. And make sure to avoid conflicts with other team members who might be using the same dataset.

In [86]:
PROJECT_ID = GCP_PROJECTS[0]
DATASET_ID = "kaggle_team"
LOCATION = "us-central1" # Please don't change this location.
#This is where you will get your data from for the lab
SOURCE_PROJECT = "kaggleworkshops"
SOURCE_DATASET = "fraud_detection"

print("Source Project: "+SOURCE_PROJECT)
print("Source Dataset: "+SOURCE_DATASET)
print("My Project: "+PROJECT_ID)
print("My Dataset: "+DATASET_ID)
print("Dataset Location: "+LOCATION)

Source Project: kaggleworkshops
Source Dataset: fraud_detection
My Project: euphoric-quanta-238417
My Dataset: kaggle_team
Dataset Location: us-central1


#### Create dataset if it does not yet exist

In [87]:
!bq mk --location=$LOCATION $DATASET_ID

BigQuery error in mk operation: Dataset 'euphoric-quanta-238417:kaggle_team'
already exists.


#### Required libraries

In [88]:
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest
import time
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# Construct a BigQuery client object.
client = bigquery.Client()

Define a function which runs a given sql statement in BQ and returns a datafram with the results. This function will be used throughout the lab.

In [89]:
def bq_query(sql, async_flag=False):
    """
    If `async_flag` set to False (default), 
        returns the query results for `sql` as a Pandas DataFrame, 
    Else, submits query asynchronously and returns nothing.
    """
    
    # Try dry run before executing query to catch any errors
    try:
        job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
        dry_run_job = client.query(sql, job_config=job_config)
    except BadRequest as err:
        print(err)
        return
        
    job_config = bigquery.QueryJobConfig()
    df = client.query(sql, job_config=job_config)
    
    if async_flag:
        return f"Query sent asynchronously for job id: {df.job_id}"
    else:
        df = df.result() #wait for query to finish running
        return df.to_dataframe()

## BigQuery ML basics <a id="basics"></a>

#### Examine given data <a id="select"></a>

In [90]:
sql_select = f"""
SELECT 
  *
FROM
  `{SOURCE_PROJECT}.{SOURCE_DATASET}.customers`
LIMIT 1000
"""

df = bq_query(sql = sql_select)

df.head()

,CUSTOMER_ID,x_customer_id,y_customer_id
0,9057108040646743,93.421400,61.396596
1,3373084462462294,94.737059,73.085581
2,9620005159501228,59.102690,65.917647
3,8355681376485783,38.033518,14.780868
4,6300457835724066,4.680635,97.073144


In [91]:
sql_select = f"""
SELECT 
  *
FROM
  `{SOURCE_PROJECT}.{SOURCE_DATASET}.terminals`
LIMIT 1000
"""

df = bq_query(sql = sql_select)

df.head()

,TERMINAL_ID,x_terminal_id,y_terminal__id
0,62052767,98.837384,10.204481
1,75282545,97.645947,46.865120
2,34460151,95.608363,64.399020
3,23509032,66.017354,29.007761
4,74438020,22.741463,25.435648


In [92]:
sql_select = f"""
SELECT 
  *
FROM
  `{SOURCE_PROJECT}.{SOURCE_DATASET}.train`
LIMIT 1000
"""

df = bq_query(sql = sql_select)

df.head()

,TX_ID,TX_TS,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD
0,2ea24fa7d76367a607fc9eb3bc66f88ee6200efb,2021-07-22 07:48:17+00:00,0069305148726523,83736813,24.790000000,0
1,3568bf1bb3fe878072de43d79ba407b1b49711b3,2021-09-25 06:01:58+00:00,7705523422228805,78424763,16.930000000,0
2,90d3f5b92c5f47719ed54f4dc7c721145b8330ee,2021-06-16 22:44:09+00:00,8017346034222291,49248924,2.390000000,0
3,9ecb2a4efcbaa071e808f95c08316fec81201399,2021-06-16 02:45:28+00:00,5970038028469253,98759254,98.960000000,0
4,7d8fcaa1cd37ed797234d518a2f43a1521809197,2021-11-25 22:56:37+00:00,0986923313045181,02479938,45.930000000,0


In [93]:
sql_select = f"""
SELECT 
  *
FROM
  `{SOURCE_PROJECT}.{SOURCE_DATASET}.test`
LIMIT 1000
"""

df = bq_query(sql = sql_select)

df.head()

,TX_ID,TX_TS,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,d803188ce5ec3354e9193cd0fa84b0b0ecb474f8,2021-12-06 16:17:46+00:00,0717942689361348,00064542,62.530000000
1,c4161477f903ee0077488cf2ed40a214747614f1,2021-12-02 10:25:57+00:00,4176271867313310,00064542,66.420000000
2,634a6a51d55166b6f89da33e6f25b5e3455d66fb,2021-12-06 18:04:15+00:00,5457708212804106,00064542,84.130000000
3,f36547436cc80382ac839b5844bfd256073b7278,2021-12-01 21:31:19+00:00,6834120891750859,00064542,41.760000000
4,63b7d5db2653b777f5501ad91273d87ab334ed75,2021-12-04 18:14:23+00:00,4388867881804879,00064542,49.200000000


#### Train a BigQuery ML model <a id="train"></a>

In [94]:
MODEL_NAME = "fraud_detection"

sql_logreg_sample = f"""
CREATE OR REPLACE MODEL
 `{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}`
  OPTIONS(
    MODEL_TYPE = 'LOGISTIC_REG',
    INPUT_LABEL_COLS = ['TX_FRAUD'],
    AUTO_CLASS_WEIGHTS = TRUE,
    ENABLE_GLOBAL_EXPLAIN = TRUE
  )
AS
SELECT
 TX_TS,
 CUSTOMER_ID,
 TERMINAL_ID,
 TX_AMOUNT,
 TX_FRAUD
FROM
 `{SOURCE_PROJECT}.{SOURCE_DATASET}.train`
"""

# send asynchronously
bq_query(sql = sql_logreg_sample, async_flag=True)

'Query sent asynchronously for job id: 094abec5-a003-434e-abd2-8982e9be7b32'

Try experimenting with <A href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create">other model types</A> as well, such as:
 - [`logistic_reg`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create)
 - [`boosted_tree_classifier`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree)
 - [`dnn_classifier`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-dnn-models)
 - BUT NOT AutoML Model types for this lab. Those give excellent results, but take much longer to train (several hours) - __too long for this workshop.__

#### Check models currently being trained on BigQuery <a id="currentlytraining"></a>

This will check if the models in your project are still undergoing training.

In [102]:
sql_currentlyrunningqueries = f"""
SELECT
    job_id,
    creation_time,
    query,
    REGEXP_EXTRACT(query, r'{DATASET_ID}\.\w+') AS modelname
FROM `region-{LOCATION}`.INFORMATION_SCHEMA.JOBS_BY_USER
WHERE 
    REGEXP_CONTAINS(LOWER(query), 'create (or replace ){{0,1}}model')
    AND NOT REGEXP_CONTAINS(LOWER(query), 'information_schema')
    AND state != "DONE"
"""

print(sql_currentlyrunningqueries)

# send asynchronously
models_still_training = bq_query(sql_currentlyrunningqueries)

print(f"{len(models_still_training)} models still currently training:")
models_still_training


SELECT
    job_id,
    creation_time,
    query,
    REGEXP_EXTRACT(query, r'kaggle_team\.\w+') AS modelname
FROM `region-us-central1`.INFORMATION_SCHEMA.JOBS_BY_USER
WHERE 
    REGEXP_CONTAINS(LOWER(query), 'create (or replace ){0,1}model')
    AND NOT REGEXP_CONTAINS(LOWER(query), 'information_schema')
    AND state != "DONE"

1 models still currently training:


,job_id,creation_time,query,modelname
0,d26ea4e9-654f-45ed-a054-ac7507bb32aa,2022-06-20 18:52:45.101000+00:00,"\nCREATE OR REPLACE MODEL `euphoric-quanta-238417.kaggle_team.fraud_detection_transform`\n TRANSFORM (\n EXTRACT (hour from TX_TS) as hour,\n CUSTOMER_ID,\n TERMINAL_ID,\n TX_AMOUN...",kaggle_team.fraud_detection_transform


#### Check which models have finished training <a id="finishedtraining"></a>

In [103]:
sql_finished_queries = f"""
SELECT
    job_id,
    creation_time,
    query,
    REGEXP_EXTRACT(query, r'{DATASET_ID}\.\w+') AS modelname
FROM `region-{LOCATION}`.INFORMATION_SCHEMA.JOBS_BY_USER
WHERE 
    REGEXP_CONTAINS(LOWER(query), 'create (or replace ){{0,1}}model')
    AND NOT REGEXP_CONTAINS(LOWER(query), 'information_schema')
    AND state = "DONE"
ORDER BY creation_time DESC
"""

print(sql_finished_queries)

finished_models = bq_query(sql_finished_queries)
finished_models.head()


SELECT
    job_id,
    creation_time,
    query,
    REGEXP_EXTRACT(query, r'kaggle_team\.\w+') AS modelname
FROM `region-us-central1`.INFORMATION_SCHEMA.JOBS_BY_USER
WHERE 
    REGEXP_CONTAINS(LOWER(query), 'create (or replace ){0,1}model')
    AND NOT REGEXP_CONTAINS(LOWER(query), 'information_schema')
    AND state = "DONE"
ORDER BY creation_time DESC



,job_id,creation_time,query,modelname
0,d26ea4e9-654f-45ed-a054-ac7507bb32aa,2022-06-20 18:52:45.101000+00:00,"\nCREATE OR REPLACE MODEL `euphoric-quanta-238417.kaggle_team.fraud_detection_transform`\n TRANSFORM (\n EXTRACT (hour from TX_TS) as hour,\n CUSTOMER_ID,\n TERMINAL_ID,\n TX_AMOUN...",kaggle_team.fraud_detection_transform
1,094abec5-a003-434e-abd2-8982e9be7b32,2022-06-20 18:46:16.141000+00:00,"\nCREATE OR REPLACE MODEL\n `euphoric-quanta-238417.kaggle_team.fraud_detection`\n OPTIONS(\n MODEL_TYPE = 'LOGISTIC_REG',\n INPUT_LABEL_COLS = ['TX_FRAUD'],\n AUTO_CLASS_WEIGHTS = TRUE,...",kaggle_team.fraud_detection
2,160627fe-b9d0-401e-a180-46ae3cae6c80,2022-06-18 18:12:24.805000+00:00,"\nCREATE OR REPLACE MODEL\n `euphoric-quanta-238417.kaggle_team.fraud_detection`\n OPTIONS(\n MODEL_TYPE = 'LOGISTIC_REG',\n INPUT_LABEL_COLS = ['TX_FRAUD'],\n AUTO_CLASS_WEIGHTS = TRUE,...",kaggle_team.fraud_detection
3,2a75dd6d-48dd-4847-a55b-c60cf5a1a594,2022-06-16 03:50:55.448000+00:00,"\nCREATE OR REPLACE MODEL\n `euphoric-quanta-238417.kaggle_student.fraud_detection`\n OPTIONS(\n MODEL_TYPE = 'LOGISTIC_REG',\n INPUT_LABEL_COLS = ['TX_FRAUD'],\n AUTO_CLASS_WEIGHTS = TR...",None
4,f77283ff-39bd-4285-a1a5-d10ab0ca5fca,2022-06-15 17:16:35.631000+00:00,"\nCREATE OR REPLACE MODEL\n `euphoric-quanta-238417.kaggle_student.fraud_detection`\n OPTIONS(\n MODEL_TYPE = 'LOGISTIC_REG',\n INPUT_LABEL_COLS = ['TX_FRAUD'],\n AUTO_CLASS_WEIGHTS = TR...",None


#### Evaluate a BigQuery ML model <a id="evaluate"></a>

In [97]:
print("Evaluating Model: {}".format(MODEL_NAME))
sql_logreg_sample = f"""
SELECT 
  *
FROM
  ML.EVALUATE(
      MODEL {DATASET_ID}.{MODEL_NAME})
"""

print(sql_logreg_sample)

bq_query(sql = sql_logreg_sample)

Evaluating Model: fraud_detection

SELECT 
  *
FROM
  ML.EVALUATE(
      MODEL kaggle_team.fraud_detection)



,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.249151,0.977778,0.93346,0.397112,0.159315,0.987379


### Compare multiple models <a id="compare"></a>

In [ ]:
MODEL_LIST = ["fraud_detection", 
              "fraud_detection_transform"] #change or add models here

sql_modeleval_list = [f"SELECT \n  '{modelname}' as modelname, \n  * \nFROM \n  ML.EVALUATE(MODEL {DATASET_ID}.{modelname})"
                      for modelname 
                      in MODEL_LIST]

sql_compare_evaluation = "\n\nUNION ALL\n\n".join(sql_modeleval_list)

print(sql_compare_evaluation)

bq_query(sql_compare_evaluation)

<a id="featinfo"></a>

### ML.FEATURE_INFO to check input feature information

The ML.FEATURE_INFO function allows you to see information about the input features used to train a model. [ML.FEATURE_INFO Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-feature)

In [98]:
sql_featureinfo = f"""
SELECT
    *
FROM
    ML.FEATURE_INFO(MODEL {DATASET_ID}.{MODEL_NAME})
"""

bq_query(sql_featureinfo)

,input,min,max,mean,median,stddev,category_count,null_count,dimension
0,TX_TS,1.609459e+09,1.640996e+09,1.623886e+09,1.623901e+09,8.329633e+06,3192915,0,<NA>
1,CUSTOMER_ID,NaN,NaN,NaN,NaN,NaN,49664,0,<NA>
2,TERMINAL_ID,NaN,NaN,NaN,NaN,NaN,5000,0,<NA>
3,TX_AMOUNT,0.000000e+00,6.050700e+02,5.291142e+01,5.206000e+01,2.922117e+01,<NA>,0,<NA>


<a id="predictions"></a>

### Make predictions using BigQuery ML

In [99]:
sql_predict = f"""
SELECT
  *
FROM
  ML.PREDICT(MODEL `{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}`,
    (
    SELECT
      *
    FROM
      `{SOURCE_PROJECT}.{SOURCE_DATASET}.test`
    )
  )
"""

print(sql_predict)

predictions = bq_query(sql_predict)
predictions.head()


SELECT
  *
FROM
  ML.PREDICT(MODEL `euphoric-quanta-238417.kaggle_team.fraud_detection`,
    (
    SELECT
      *
    FROM
      `kaggleworkshops.fraud_detection.test`
    )
  )



,predicted_TX_FRAUD,predicted_TX_FRAUD_probs,TX_ID,TX_TS,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,"[{'label': 1, 'prob': 0.10071210889554656}, {'label': 0, 'prob': 0.8992878911044534}]",d803188ce5ec3354e9193cd0fa84b0b0ecb474f8,2021-12-06 16:17:46+00:00,0717942689361348,00064542,62.530000000
1,0,"[{'label': 1, 'prob': 0.1566286389468031}, {'label': 0, 'prob': 0.8433713610531969}]",c4161477f903ee0077488cf2ed40a214747614f1,2021-12-02 10:25:57+00:00,4176271867313310,00064542,66.420000000
2,0,"[{'label': 1, 'prob': 0.11056953803790275}, {'label': 0, 'prob': 0.8894304619620973}]",634a6a51d55166b6f89da33e6f25b5e3455d66fb,2021-12-06 18:04:15+00:00,5457708212804106,00064542,84.130000000
3,0,"[{'label': 1, 'prob': 0.06325088782013324}, {'label': 0, 'prob': 0.9367491121798668}]",f36547436cc80382ac839b5844bfd256073b7278,2021-12-01 21:31:19+00:00,6834120891750859,00064542,41.760000000
4,0,"[{'label': 1, 'prob': 0.06534000622627109}, {'label': 0, 'prob': 0.9346599937737289}]",63b7d5db2653b777f5501ad91273d87ab334ed75,2021-12-04 18:14:23+00:00,4388867881804879,00064542,49.200000000


<a id="exportcsv"></a>

#### Congratulations!

You've now trained a basic model and made a batch prediction using BQML. But this is just the beginning. You have many more BQML tools at your disposal that you can use to optimize and improve your model accuracy and performance. This is where your imagination and ML intuition will be critical to best your fellow competitors. Below you'll find examples of just a few techniques you can incorporate. You may also want to use Feature Engineering to create new columns that will make your model more effective. If helpful, you can write out your data to new tables or datasets in your student project.

Don't be afraid to Google (or Bing, whatever ;-) ) the BQML docs for more ideas and inspiration.

<hr><a id="feateng"></a>

## Preprocessing Functions in BigQuery ML

Below are some examples of various transformations that you can apply using BQ SQL. You can apply these transformations using a SELECT statement before passing to your model, optionally storing them in a separate table first - but likely better is to incorporate them into a __TRANSFORM__ directly within the model definition itself. This avoids needing to replicate the transformation when you perform predictions, which is handy when working with distributed teams or over a period of time. Then when making predictons you can just pass in the data in it's original form and the model will perform the transformation for you during the prediction.

Note that below examples are just to show you the syntax and how it works - they may not be all that useful as-is and may not be appropriate for this particular ML exercise.

#### TRANSFORM

You can use [TRANSFORM()](https://cloud.google.com/bigquery-ml/docs/bigqueryml-transform) with any standard SQL function or any ML [preprocessing function](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions).

Here's an example of a simple TRANSFORM. The lower SELECT captures the columns from the raw data. These are captured by the TRANSFORM statement toward the top and passed into the model as the features.

In [100]:
MODEL_NAME = "fraud_detection_transform"

sql_logreg_sample = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}`
  TRANSFORM (
    EXTRACT (hour from TX_TS) as hour,
     CUSTOMER_ID,
     TERMINAL_ID,
     TX_AMOUNT,
     TX_FRAUD
     )
  OPTIONS(
    MODEL_TYPE = 'dnn_classifier',
    INPUT_LABEL_COLS = ['TX_FRAUD'],
    AUTO_CLASS_WEIGHTS = TRUE,
    ENABLE_GLOBAL_EXPLAIN = TRUE
  )
AS
SELECT
*
FROM
 `{SOURCE_PROJECT}.{SOURCE_DATASET}.train`
"""

# send asynchronously
bq_query(sql = sql_logreg_sample, async_flag=True)

'Query sent asynchronously for job id: d26ea4e9-654f-45ed-a054-ac7507bb32aa'

<a id="extractdates"></a>

#### EXTRACT: Extract dates and time

[EXTRACT from DATE documentation](https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions#extract)  
[EXTRACT from TIMESTAMP documentation](https://cloud.google.com/bigquery/docs/reference/standard-sql/timestamp_functions#extract)

In [ ]:
sql_datetime = f"""
    SELECT
        TX_TS,
        EXTRACT( year FROM TX_TS) AS year,
        EXTRACT( month FROM TX_TS) AS month,
        EXTRACT( dayofweek FROM TX_TS) AS dayofweek,
        EXTRACT( day FROM TX_TS) AS day,
    FROM
        {SOURCE_PROJECT}.{SOURCE_DATASET}.train
    LIMIT 10
    """

bq_query(sql_datetime)

#### ML.QUANTILE_BUCKETIZE: Bucketize your numeric feature into bins
[ML.QUANTILE_BUCKETIZE Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions#quantile_bucketize)

In [ ]:
sql_bucketize = f"""
SELECT 
    TX_AMOUNT,
    ML.QUANTILE_BUCKETIZE(
      TX_AMOUNT,
      100 # number of buckets
    ) OVER() AS bucket_tx_amount,
FROM
    {SOURCE_PROJECT}.{SOURCE_DATASET}.train
LIMIT 10
"""

bq_query(sql_bucketize)

<a id="polynomialexpand"></a>

#### ML.POLYNOMIAL_EXPAND: Combine numeric features

[ML.POLYNOMIAL_EXPAND Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions#mlpolynomial_expand)

In [ ]:
sql_polynomialexpand = f"""
SELECT
    TX_AMOUNT,
    int_rate,
    ML.POLYNOMIAL_EXPAND(
      STRUCT(loan_amnt, int_rate), 
      2
    ) AS x
FROM
    {SOURCE_PROJECT}.{SOURCE_DATASET}.train
LIMIT 3
"""

bq_query(sql_polynomialexpand)

<a id="featurecross"></a>

#### ML.FEATURE_CROSS: Combine categorical features

[ML.FEATURE_CROSS Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions#mlfeature_cross)

In [ ]:
sql_featurecross = f"""
SELECT
    grade,
    sub_grade,
    ML.FEATURE_CROSS(
      STRUCT(grade, sub_grade), 
      2
    ) AS x
FROM
    {SOURCE_PROJECT}.{SOURCE_DATASET}.train
LIMIT 3
"""

bq_query(sql_featurecross)

<a id="featselection"></a>
<hr>

## Feature Selection

<a id="featimportance"></a>

### ML.FEATURE_IMPORTANCE for Boosted Tree models

_Note: This function is only available for Boosted Tree models (`boosted_tree_classifier` or `boosted_tree_regressor`)._

The ML.FEATURE_IMPORTANCE function allows you to see feature importance score, which indicates how useful or valuable each feature was in the construction of the Boosted Tree model during training. [ML.FEATURE_IMPORTANCE Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-importance#mlfeature_importance_function)

In [ ]:
MODEL_NAME = "MY_BOOSTED_TREE_MODEL_NAME"

sql_featureimportance = f"""
SELECT
    *
FROM
    ML.FEATURE_IMPORTANCE(MODEL {DATASET_ID}.{MODEL_NAME})
"""

bq_query(sql_featureimportance)

## Submit Your Results <a id="submit"></a>

Once you have a reasonble candidate model, you're ready to submit your predications to Kaggle for evaluation and scoring. Don't be shy! You can make up to 20 (!!) submissions and they all count. So feel completely free to iterate on this as much as you want to ~~destroy your competitors~~ encourage everyone to up their game!

### Export predictions for Kaggle as a CSV file

In [ ]:
# Make sure to set MODEL_NAME to the correct model name that you want to submit predictions for.
MODEL_NAME = "fraud_detection"

sql_predict = f"""
SELECT
  TX_ID,
  probs.prob as TX_FRAUD
FROM
  ML.PREDICT(MODEL `{PROJECT_ID}.{DATASET_ID}.{MODEL_NAME}`,
    (
    SELECT
      *
    FROM
      `{SOURCE_PROJECT}.{SOURCE_DATASET}.test`
    )
  ),
  UNNEST (predicted_TX_FRAUD_probs) as probs
  WHERE label = 1
"""

print(sql_predict)

submission = bq_query(sql_predict)
submission.head()
#save as CSV
submission.to_csv(f"submission_{MODEL_NAME}.csv", index=False)

The predictions are now saved to a CSV file. To download the file to your notebook server. Locate your file in the left-menu in JupyterLab, right-click on it and click "Download". You can now make a submission to the <A href="http://www.kaggle.com/c/cloudday2022">Kaggle Competition</A> with the file.